In [10]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py transformers torch einops pypdf onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 5.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.18.1
    Uninstalling onnxruntime-1.18.1:
      Successfully uninstalled onnxruntime-1.18.1

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os 
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = os.environ['LANGCHAIN_TRACING_V2'] = 'true'
LANGCHAIN_ENDPOINT = os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
LANGCHAIN_API_KEY = os.environ['LANGCHAIN_API_KEY']
FIRECRAWL_API_KEY = os.environ['FIRECRAWL_API_KEY']

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
from langchain_community.vectorstores import Chroma

# Load the embedding model and tokenizer
model_name = "nomic-ai/nomic-embed-text-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Function to generate embeddings using the loaded model
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings[0].cpu().numpy().tolist()  # Convert ndarray to list

# Wrapper class to use with Chroma
class CustomEmbedding:
    def embed_documents(self, texts):
        return [embed_text(text) for text in texts]

    def embed_query(self, text):
        return embed_text(text)

# Instantiate the embedding class
custom_embedding = CustomEmbedding()

# Add documents with embeddings to the vectorDB using the embedding class
vectorstore = Chroma.from_documents(
    documents=pages,
    collection_name="rag-chroma",
    embedding=custom_embedding,  # Use the embedding class instance
)

retriever = vectorstore.as_retriever()

<All keys matched successfully>


AttributeError: module 'chromadb' has no attribute 'config'

In [3]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3.1:latest", format="json", temperature=0)

/var/folders/86/7vl_g9y936j10tb2scb6w_jr0000gn/T/ipykernel_8991/3253887872.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1:latest", format="json", temperature=0)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.document_loaders import PyPDFLoader

docs = ('/Users/tt-josue-sanchez/Downloads/Documents from Sanchez Josue.pdf')

loader = PyPDFLoader(docs)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

print(f"{pages[0].metadata}\n")
print(pages[0].page_content)


SHIP: ITEM NO: 000-00-000
COAR: PCN:
BWST FILE NO: CMP:
REVISED:
USS TEST (LHD 0) 
69
000-000
16 JUNE 1945 PLANNER:
AB0-U04K
NONE
MUSK,ELON
SSP: XXXX-LHD0-XXXXX25-S001 1 of 
2 ITEM NO: 000-00-000
EXPORT CONTROLLED DATA: TEST SPEC NO CUI.
1. SCOPE:
1.1  Title: No. 2 Emergency Diesel Generator (EDG) Jacket Water Pump Gear
Backlash Adjustment; accomplish
1.2  Location of Work:
1.2.1  EMERGENCY Gen Rm No. 0 (0-000-0-E)
1.3  Identification:
1.3.1  Quantity (One EA), EDG, No. 2, Mfr: TEST COMPANY Engine,
Mfr Dwg: 1000001, Mfr ID: 000C, No. of Cylinder: 16, Bore: 9
Inch, Stroke: 10.250 Inch, Speed: 1000 RPM, Power Rating: 3,000
HP at 1000 RPM Turbocharged, Rotation Viewed from Flywheel: CCW,
Compression 11.5 to One, APL: 664010028A
1.3.1.1  Quantity (One EA), Jacket Water Pump, Attached, 
Centrifugal, Capacity: 800 GPM, Discharge Pressure: 35.5 PSIG,
Suction Pressure: 3 PSIG, Temperature: 180 Degrees F, Speed: 
900 RPM, Figure 69-1 of 2.2
2. REFERENCES:
2.1  Standard Items, (FY25)
2.2  S0000-

In [11]:
from transformers import AutoModel, AutoTokenizer
import torch
from langchain_community.vectorstores import Chroma

# Load the embedding model and tokenizer
model_name = "nomic-ai/nomic-embed-text-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Function to generate embeddings using the loaded model
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings[0].cpu().numpy().tolist()  # Convert ndarray to list

# Wrapper class to use with Chroma
class CustomEmbedding:
    def embed_documents(self, texts):
        return [embed_text(text) for text in texts]

    def embed_query(self, text):
        return embed_text(text)

# Instantiate the embedding class
custom_embedding = CustomEmbedding()

# Add documents with embeddings to the vectorDB using the embedding class
vectorstore = Chroma.from_documents(
    documents=pages,
    collection_name="rag-chroma",
    embedding=custom_embedding,  # Use the embedding class instance
)

retriever = vectorstore.as_retriever()

<All keys matched successfully>


AttributeError: module 'chromadb' has no attribute 'config'

In [63]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Define the prompt template
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assesing 
    relevance of a retrieved document to a user question. If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n 
    Give a binary score 'yes' or 'no' score to indicate whether the docuemnt is relevant to the question. \n 
    Providde the binary score as a JSON with a single key 'score' and no premable or explaination.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document* \n\n {document} \n\n
    Here is the user question* {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
    input_variables=["question", "document"]
)

# Chain the prompt, LLM, and output parser together
retrieval_grader = prompt | llm | JsonOutputParser()

# Define the user question
question = "What is Tokyo Techies?"

# Retrieve documents related to the question
docs = retriever.invoke(question)

# Get the content of the second retrieved document
doc_txt = docs[1].page_content  # Use page_content instead of page_context

# Grade the relevance of the document
result = retrieval_grader.invoke({"question": question, "document": doc_txt})

# Print the result
print(result)

{'score': 'yes'}


In [81]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assitant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
input_variables=["question", "document"]
)

#Post processing
def format_doc(docs):
    return"\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm | StrOutputParser()

#Run

question = "quien es duc doba?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

{ "Duc DoBa"  : "FOUNDER & CEO" }


In [79]:
### Hallucination Grader
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assesing whether
    an answer is grounded in / supported by a set of facts. Give binary scores 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preambel or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n -------- \n
    {documents}
    \n -------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""",
input_variables=["generation", "document"]
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": doc, "generation": generation})

{'score': 'no'}